In [82]:
import numpy as np
import scipy.special as sp
from numpy import linalg as al
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import csv

from itertools import combinations
from random import random , choice

import winsound

#conjuntos de nodos

all_nodes = ['C3', 'C4', 'CZ', 'EMG', 'F3', 'F4', 'F7', 'F8', 'FP1', 'FP2', 'FZ', 'LOG', 'O1', 'O2', 'P3', 'P4', 'PZ', 
             'ROG', 'T3', 'T4', 'T5', 'T6']

frontal=['F3', 'F4', 'F7', 'F8', 'FP1', 'FP2', 'FZ','LOG','ROG']
central=['C3', 'C4', 'CZ']
izquierdo = ['C3', 'F3', 'F7', 'FP1', 'LOG', 'O1','P3','T3',  'T5']
medio = ['CZ','FZ','PZ']
derecho = ['C4','F4', 'F8', 'FP2',  'O2', 'P4', 'ROG', 'T4', 'T6']

zonas_=['all_nodes','izquierdo','derecho','medio','izquierdo+medio','derecho+medio','izquierdo+derecho']

#-------------------------------------------------------

#funciones varias

def graph_from_csv(filename):
    df = pd.read_csv(filename, index_col=0)
    return nx.from_pandas_adjacency(df)

def positiv(graph):
    for i in graph.nodes:
        for j in graph.nodes:
            if i !=j:
                graph[i][j]['weight'] = abs(graph[i][j]['weight'])
            else:
                graph[i][j]['weight'] = 0
    return(graph)

def random_weighted_graph(the_nodes):
    G = nx.Graph()
    G.add_nodes_from(the_nodes)
    all_edges = combinations(the_nodes, 2)
    elist = [(e[0], e[1], random()) for e in all_edges]
    G.add_weighted_edges_from(elist)
    return G

def selected_graph(graph, param, bigger_than=True):
    vs = list(graph.nodes())
    n = graph.order()
    out = nx.Graph()
    out.add_nodes_from(vs)
    if bigger_than:
        out.add_edges_from([(vs[i], vs[j]) for i in range(n) for j in range(n) if i != j
                            and graph[vs[i]][vs[j]]['weight'] > param])
    else:
        out.add_edges_from([(vs[i], vs[j]) for i in range(n) for j in range(n) if i != j
                            and graph[vs[i]][vs[j]]['weight'] < param])
    return out

def eigenvalores(A):
        av=list(al.eigh(A))
    
        av[0]=av[0].astype(float)
        av[1]=av[1].astype(float)
    
        for i in range(len(av[0]-1)):
            av[0][i]=round(float(av[0][i]),5)
    
        return av[0]
    
def eigenvectores(A):
        av=list(al.eigh(A))
    
        av[1]=av[1].astype(float)
    
        for i in range(len(av[1]-1)):
            for j in range(len(av[1][i]-1)):
                av[1][i][j]=round(float(av[1][i][j]),3)
    
        return av[1].T
    
def Fiedler(G):
        lap=nx.laplacian_matrix(G)
        print("valor Fiedler")
        print(eigenvalores(lap.A)[1])
        print("vector Fiedler")
        print(eigenvectores(lap.A)[1])
        print("los eigenvalores")
        print(eigenvalores(lap.A))
    
    # Función genérica para el cociente de Rayleigh para una matriz y un vector,
    
def rayleigh(A,v):
        numerador = np.dot(np.dot(A,v),v)
        denominador = np.dot(v,v)
        S = numerador/denominador
        return S
    
def peso_grafica(G):
    long=len(G.nodes)
    peso=sum([list(G.degree(weight='weight'))[k][1]/2 for k in range(long)])
    return(peso)
    
    
def vector_candidato(G): #un vector sencillo candidato a tener cociente de Rayleigh pequeño
    grados=[list(G.degree(weight='weight'))[k][1] for k in range(len(G))]

    nodo_uno=list(G.nodes)[np.argmin(np.array(grados))]

    grados1=[list(G.subgraph(set(G.nodes)-{nodo_uno}).degree(weight='weight'))[k][1] for k in range(len(G)-1)]
    #e2=np.where(np.array(grados1)==min(grados1))
    nodo_dos=list(G.nodes)[np.argmin(np.array(grados1))]

    v=[0]*len(G)
    v[np.argmin(np.array(grados))]=1
    v[np.argmin(np.array(grados1))]=-1
    return(v)

def un_maximo_local(G):
    maximal_ac = conectividad_algebraica(G)
    print(maximal_ac)
    
    subgrafica=G
    for i in range(1,len(G)):
        size=len(subgrafica)-1
        print("\r",size," ",end="")
        if size<maximal_ac:
            break
        subgrafica = G.subgraph(quitar_uno_express(subgrafica))
        ac = conectividad_algebraica(subgrafica)
        if ac>maximal_ac: 
            maximal_ac = ac
            grafica_optima = subgrafica
            print(maximal_ac)
    subgrafica = grafica_optima
    nodos_nuevos=set(grafica_optima.nodes)
    print("\r","    \n","\rde bajada candidato a máximo en",nodos_nuevos)
    print("con valor Fiedler igual a",maximal_ac)
    print("número de nodos", len(grafica_optima))
    for i in range(len(grafica_optima),len(G)-1):
        subgrafica=poner_uno_express(G,subgrafica)
        if conectividad_algebraica(subgrafica)>maximal_ac:
            maximal_ac = conectividad_algebraica(subgrafica)
            grafica_optima = subgrafica
            print(i,maximal_ac)
            
    nodos_nuevos=set(grafica_optima.nodes)
    print("\r","    \n","\rcandidato a máximo en",nodos_nuevos)
    print("con valor Fiedler igual a",maximal_ac)
    print("número de nodos", len(grafica_optima))
    return ([grafica_optima,maximal_ac])

def pares_vetados(G,cota): #pares de nodos que no pueden estar en subgráfica con valor Fiedler mayor a cota
    lap = nx.laplacian_matrix(G)
    pares = [tuple([aristas[0],aristas[1]]) for aristas in G.edges if aristas[0]!=aristas[1]]
    vetados = []
    for pareja in pares:
        v = [0]*len(G.nodes)
        for k in range(len(G.nodes)):
            if list(G.nodes)[k]==pareja[0]:
                v[k]=-1
            if list(G.nodes)[k]==pareja[1]:
                v[k]=1
        cociente = rayleigh(lap.A,v)
        print(cociente)
        if cociente<cota:
            vetados.append(pareja)
    return(vetados)
    
''' 
def subgrafica_optima_ac(G):#Dada una gráfica con pesos, regresa la subgráfica con máximo valor Fiedler
    

    return(G.subgraph(nodos_selectos))    
'''

#--------------------------------------------------------    

#funciones particulares para explorar valores Fiedler de subgráficas de los datos del PSG

def quitar_uno(G):
    print("El valor Fiedler de toda la gráfica es",round(conectividad_algebraica(positiv(G)),5))
    print("\n")

    valores_nodos=dict(zip(G.nodes,eigenvectores(lap.A)[1]))
    print("El eigenvector Fiedler correspondiente es",valores_nodos)

    todos=set(G.nodes)
    nuevas=list([round(conectividad_algebraica(positiv(G).subgraph(todos-{nodo})),5) for nodo in list(todos)])
    print("\n")
    print("Valores Fiedler, quitando el nodo que se indica: ")
    
    d=dict(zip(list(todos),nuevas))
    print(d)
    print("\n")
    
    i = 0
    for nodo in list(todos):
        #print(nodo)
        if nuevas[i] == max(nuevas):
            print(list(todos)[i],"You are the weakest link")
            print("Quitando",list(todos)[i],"el valor Fiedler alcanza su máximo, que es",nuevas[i])
            break
        i = i+1
    return()
    #nx.draw_circular(positiv(G).subgraph(todos-{nodo}),with_labels=True)  
    
def quitar_uno_express(G):
    todos = tuple(G.nodes)
    reducidos = [conectividad_algebraica(positiv(G).subgraph(set(todos)-{nodo})) for nodo in todos]
    weak_link={todos[np.argmax(reducidos)]}
    nuevos_nodos = tuple(set(todos)-weak_link)  
    
    return(nuevos_nodos)

def poner_uno_express(G,S):
    sobrantes=tuple(set(G.nodes)-set(S.nodes))
    graficas_aumentadas = [G.subgraph(tuple(set.union(set(S.nodes),{nodo_extra}))) for nodo_extra in sobrantes]
    vf_aumentadas = [conectividad_algebraica(aumentadas) for aumentadas in graficas_aumentadas]
    optimo = np.argmax(vf_aumentadas)
    
    return(graficas_aumentadas[optimo])    
    
def regiones_especificas(G):
    subs=list()

    [subs.append(positiv(G).subgraph(zona)) for zona in zonas] 

    print("\n\n Valores Fiedler por regiones\n")

    for i in range(len(zonas)):
        print(zonas_[i],"         ",round(conectividad_algebraica(subs[i]),5))

def conectividad_algebraica(G):
    laplace=nx.laplacian_matrix(G)
    return(eigenvalores(laplace.A)[1])

In [83]:
    G = positiv(graph_from_csv('./variaciones/CorrelacionesREM/correls_JANA.csv'))
cota=un_maximo_local(G)[1]
pares_nene=pares_vetados(G,cota)
len(pares_nene)

6.8776
 21  7.78634
 20  8.28554
     
de bajada candidato a máximo en {'O1', 'T4', 'F3', 'T5', 'EMG', 'CZ', 'FP2', 'C3', 'P4', 'T3', 'F7', 'P3', 'PZ', 'T6', 'FP1', 'F4', 'F8', 'O2', 'FZ', 'C4'}
con valor Fiedler igual a 8.28554
número de nodos 20
     
candidato a máximo en {'O1', 'T4', 'F3', 'T5', 'EMG', 'CZ', 'FP2', 'C3', 'P4', 'T3', 'F7', 'P3', 'PZ', 'T6', 'FP1', 'F4', 'F8', 'O2', 'FZ', 'C4'}
con valor Fiedler igual a 8.28554
número de nodos 20
15.572892848485148
16.412945780936486
14.461683289669777
15.187700369681064
16.42262974044049
12.907038955945152
16.278967571849314
13.204603719055974
13.108388184754851
15.220146471710997
11.306930552720843
16.23234604316184
16.231098066365345
16.45470755986404
16.12989223418942
16.32575724203709
12.207499195679688
15.656518735212762
16.24644585201049
16.706631765415498
16.098163532500852
15.891629620035598
14.064545336030903
14.421292871464942
15.984208726391758
12.196170559286413
15.86330555549215
12.465604556447374
12.375397156187566
14.

0

[('C3', 'EMG'),
 ('C3', 'FP2'),
 ('C3', 'P3'),
 ('EMG', 'F3'),
 ('EMG', 'F8'),
 ('EMG', 'FP1'),
 ('EMG', 'FP2'),
 ('EMG', 'LOG'),
 ('EMG', 'P3'),
 ('EMG', 'ROG'),
 ('F3', 'FP2'),
 ('F3', 'P3'),
 ('F4', 'P3'),
 ('F8', 'FP2'),
 ('F8', 'LOG'),
 ('F8', 'P3'),
 ('FP1', 'P3'),
 ('FP2', 'LOG'),
 ('FP2', 'P3'),
 ('FP2', 'ROG'),
 ('LOG', 'P3'),
 ('LOG', 'ROG'),
 ('P3', 'ROG')]

In [4]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesREM/correls_RLMN.csv'))
list(G.edges)

[('C3', 'C3'),
 ('C3', 'C4'),
 ('C3', 'EMG'),
 ('C3', 'F3'),
 ('C3', 'F4'),
 ('C3', 'F7'),
 ('C3', 'F8'),
 ('C3', 'FP2'),
 ('C3', 'FZ'),
 ('C3', 'LOG'),
 ('C3', 'O1'),
 ('C3', 'O2'),
 ('C3', 'P3'),
 ('C3', 'P4'),
 ('C3', 'PZ'),
 ('C3', 'ROG'),
 ('C3', 'T3'),
 ('C3', 'T4'),
 ('C3', 'T5'),
 ('C3', 'T6'),
 ('C4', 'C4'),
 ('C4', 'EMG'),
 ('C4', 'F3'),
 ('C4', 'F4'),
 ('C4', 'F7'),
 ('C4', 'F8'),
 ('C4', 'FP2'),
 ('C4', 'FZ'),
 ('C4', 'LOG'),
 ('C4', 'O1'),
 ('C4', 'O2'),
 ('C4', 'P3'),
 ('C4', 'P4'),
 ('C4', 'PZ'),
 ('C4', 'ROG'),
 ('C4', 'T3'),
 ('C4', 'T4'),
 ('C4', 'T5'),
 ('C4', 'T6'),
 ('EMG', 'EMG'),
 ('EMG', 'F3'),
 ('EMG', 'F4'),
 ('EMG', 'F7'),
 ('EMG', 'F8'),
 ('EMG', 'FP2'),
 ('EMG', 'FZ'),
 ('EMG', 'LOG'),
 ('EMG', 'O1'),
 ('EMG', 'O2'),
 ('EMG', 'P3'),
 ('EMG', 'P4'),
 ('EMG', 'PZ'),
 ('EMG', 'ROG'),
 ('EMG', 'T3'),
 ('EMG', 'T4'),
 ('EMG', 'T5'),
 ('EMG', 'T6'),
 ('F3', 'F3'),
 ('F3', 'F4'),
 ('F3', 'F7'),
 ('F3', 'F8'),
 ('F3', 'FP2'),
 ('F3', 'FZ'),
 ('F3', 'LOG'),
 ('F3', 

In [9]:
a=[1,2,3,4,5,6,7,8,9,0,1,2,3,4,5]
b=[3,5,7]
for k in b:
    a[k]='P'
a
    
    

[1, 2, 3, 'P', 5, 'P', 7, 'P', 9, 0, 1, 2, 3, 4, 5]

In [35]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesREM/correls_JANA.csv'))
maximal_ac=round(nx.algebraic_connectivity(positiv(G)),5)
subgrafica_optima_ac_(G)



Revisando los de tamaño  21
por revisar (todos) 22
por revisar actualizado 22
con primer criterio son 22

Hasta ahora el máximo es  7.7863404765898565
Se alcanza en ('C3', 'C4', 'CZ', 'EMG', 'F3', 'F4', 'F7', 'F8', 'FP1', 'FP2', 'FZ', 'O1', 'O2', 'P3', 'P4', 'PZ', 'ROG', 'T3', 'T4', 'T5', 'T6')

Revisando los de tamaño  20
por revisar (todos) 231
por revisar actualizado 231
con primer criterio son 223

Hasta ahora el máximo es  8.285543801477962
Se alcanza en ('C3', 'C4', 'CZ', 'EMG', 'F3', 'F4', 'F7', 'F8', 'FP1', 'FP2', 'FZ', 'O1', 'O2', 'P3', 'P4', 'PZ', 'T3', 'T4', 'T5', 'T6')

Revisando los de tamaño  19
por revisar (todos) 1540
por revisar actualizado 1408
con primer criterio son 1227

Revisando los de tamaño  18
por revisar (todos) 7315
por revisar actualizado 5012
con primer criterio son 4881

Revisando los de tamaño  17
por revisar (todos) 26334
por revisar actualizado 15505
con primer criterio son 15158

Revisando los de tamaño  16
por revisar (todos) 74613
por revisar actua

In [32]:
G = positiv(graph_from_csv('./variaciones/CorrelacionesREM/correls_JGMN.csv'))
print(round(nx.algebraic_connectivity(positiv(G)),5),"\n")
optima_JGMN=subgrafica_optima_ac_(G)
nx.draw_circular(optima_JGMN,with_labels=True)     

0.26188 


Revisando los de tamaño  21
por revisar (todos) 22
por revisar actualizado 22
con primer criterio son 22

Hasta ahora el máximo es  1.7743807200770194
Se alcanza en ('C3', 'C4', 'CZ', 'EMG', 'F4', 'F7', 'F8', 'FP1', 'FP2', 'FZ', 'LOG', 'O1', 'O2', 'P3', 'P4', 'PZ', 'ROG', 'T3', 'T4', 'T5', 'T6')

Revisando los de tamaño  20
por revisar (todos) 231
por revisar actualizado 231
con primer criterio son 213

Hasta ahora el máximo es  1.9170502370718792
Se alcanza en ('C3', 'C4', 'CZ', 'EMG', 'F4', 'F7', 'F8', 'FP1', 'FP2', 'FZ', 'LOG', 'O1', 'O2', 'P3', 'P4', 'PZ', 'ROG', 'T3', 'T5', 'T6')

Revisando los de tamaño  19
por revisar (todos) 1540
por revisar actualizado 1316
con primer criterio son 1313

Revisando los de tamaño  18
por revisar (todos) 7315
por revisar actualizado 5797
con primer criterio son 5775

Revisando los de tamaño  17
por revisar (todos) 26334
por revisar actualizado 18955
con primer criterio son 18870

Revisando los de tamaño  16
por revisar (todos) 74613
por

ValueError: max() arg is an empty sequence

In [44]:
v=[0]*22
v[0]=1
type(v)

list